# The epidemic threshold of an ER network

Do *all* diseases lead to epidemics? Intuitively it seems quite unlikely: a really uninfectious disease (one with a small value of $\beta$), or a disease that didn't stay infectious for very long (one with a large value of $\alpha$) -- or both -- would seem to be candidates for diseases that don't become epidemic. In the first case, the disease is hard to pass from infected to susceptible individuals; in the latter, it has only a small time window in which to do so before the host individual is removed. One might also expect that a really sparse network would fail to transmit the disease well, since an infectd node would have few neighbours to infect.

It turns out that, for some networks at least, there's a well-defined value of infectiousness that, when exceeded, causes epidemic outbreaks. This is referred to as the **epidemic threshold** (also sometimes called the **percolation threshold** for reasons that'll become clear when we discuss percolation theory).

In [1]:
# simulation and maths packages
import numpy
import networkx
from epyc import HDF5LabNotebook, ParallelLab, RepeatedExperiment
from epydemic import StochasticDynamics, SIR, ERNetwork

# display and interaction
from IPython.display import display, Math
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import dash_core_components as dcc
import dash_html_components as html

In [2]:
# start a local lab
nb = HDF5LabNotebook('datasets/epidemic-threshold.h5', create=True)
lab = ParallelLab(nb, cores=10)

## What is the epidemic threshold?

## Exploring the epidemic threshold

Let's see how the epidemic threshold manifests itself. To do this we'll run an epidemic disease process over a random ER network, keeping the removal (recovery) rate constant and varying the infection rate. We expect that the size of epidemic will shift dramatically, rather than steadily, as the infection rate increases. This **phase transition** -- from small numbers of infected individuals to essentially everyone being infected -- occurs when $\beta$ cross the epidemic threshold, which we'll denote $\beta_{crit}$.

We need a network that is "large enough" to deonstrate the effect we're looking for. (We'll explore different sizes a little later.) For now, we can start with a modest 1000-node network with $\langle k \rangle = 8$.

In [3]:
# ER network parameters
N = 1000
kmean = 8
phi = kmean / N

# the theoretical epidemic (percolation) threshold for ER networks 
Tcrit = 1 / kmean

### Running the experiment

In [4]:
if not nb.already('epidemic-threshold-er'):
    # create a result set for the simulation
    rs = nb.addResultSet('epidemic-threshold-er')

    # set the parameters
    lab[ERNetwork.N] = N
    lab[ERNetwork.PHI] = phi
    lab[SIR.P_INFECT] = numpy.linspace(0.0, 1.0, num=250)
    lab[SIR.P_REMOVE] = 1.0
    lab[SIR.P_INFECTED] = 0.005

    # create an experiment combining the model with a network class 
    e = StochasticDynamics(SIR(), ERNetwork())

    # run the experiment over the parameter space 
    lab.runExperiment(RepeatedExperiment(e, 10))
    rs.finish()
    nb.commit()

    # retrieve the results ready for analysis
    df = rs.dataframe()

In [5]:
# load the simulation results
rs = nb.select('epidemic-threshold-er')
df = rs.dataframe()

### The results

In [6]:
app = JupyterDash('epidemic-threshold-er')

app.layout = html.Div([
    dcc.Graph(id='epidemic-threshold-er-plot'),
    dcc.Checklist(id='epidemic-threshold-er-include',
                  options=[dict(label='Include zero and non-epidemic outbreaks', value='ALL')], value=['ALL'])
])

@app.callback(
    Output('epidemic-threshold-er-plot', 'figure'),
    [Input('epidemic-threshold-er-include', 'value')]
)
def plot_epidemic_threshold(params):
    if 'ALL' in params:
        pdf = df
    else:
        pdf = df[df[SIR.REMOVED] >= N * 0.01]
    mdf_x = pdf[SIR.P_INFECT].unique()
    mdf_y = [ pdf[pdf[SIR.P_INFECT] == pInfect][SIR.REMOVED].mean() for pInfect in mdf_x ]
            
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pdf[SIR.P_INFECT], y=pdf[SIR.REMOVED],
                             mode='markers', marker=dict(color='powderblue'),
                             name='Individual samples'))
    fig.add_trace(go.Scatter(x=mdf_x, y=mdf_y,
                             mode='lines', line = dict(color='blue'),
                             name='Mean epidemic size'))
    fig.add_trace(go.Scatter(x=[Tcrit, Tcrit], y=[0.0, pdf[SIR.REMOVED].max()],
                             mode='lines', line=go.scatter.Line(color='gray'),
                             showlegend=False))
    
    return fig
            
app.run_server(mode='inline')